In [ ]:
import pandas as pd
import numpy as np

## Ajout odnnées de tmdb

Pour ajouter des données telles que la popularité, les pays de production, les affiches de films, nous avons du nous servir du tmdb.csv

In [ ]:
df_tmbd = pd.read_csv("tmdb_full.csv")

In [ ]:
df_tmbd = df_tmbd.drop(columns = [
    "adult",
    "backdrop_path",
    "budget",
    "genres",
    "homepage",
    "id",
    "original_language",
    "original_title",
    "overview",
    "release_date",
    "revenue",
    "runtime",
    "spoken_languages",
    "tagline",
    "title",
    "video",
    "vote_average",
    "vote_count",
    "production_companies_name",
    "production_companies_country"])

Merge nos nouvelles données avec le df_film 

In [ ]:
df_film = pd.read_csv("df_film.csv")
df_merge = df_film.merge(df_tmbd, left_on = "id_film", right_on = "imdb_id", how = "left")
df_merge = df_merge.drop(columns = ["Unnamed: 0.2","Unnamed: 0.1", "Unnamed: 0", "tconst"])

## Créer des colonnes supplémentaires

Créer la colonne décénie

In [ ]:
df_film["decenie"] = ((df_film['année'] // 10) * 10).astype(str)

## Remplacer des valeurs manquantes

Transformer les valeurs nulles en format compréhensible par python. Dans notre df actuellement les valeurs nulles sont des strings '\\N'

In [ ]:
df_film = df_film.replace('\\N', np.nan)

### Durée (temps_minutes)

Ici, on veut remplacer les valeurs nulles par la moyenne des autres films de la même décénie

Converti le temps (str) en int en contournant les valeurs nulles (fonctionne uniquement avec un type compris par python)

In [ ]:
df_film['temps_minutes'] = df_film['temps_minutes'].astype('Int64')

Cacul de la moyenne par décénie

In [ ]:
moy_decenie = df_film.groupby("decenie")["temps_minutes"].mean()

fonction qui remplace les valeurs nulles par la moyenne de la décénie à laquelle il appartient

In [ ]:
def replacena_mean(row):
    if str(row["temps_minutes"]) == "<NA>":
        row["temps_minutes"] = moy_decenie.loc[row["decenie"]]
    return row

In [ ]:
# Appliquer cette fonction
df_film = df_film.apply(replacena_mean, axis=1)